In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import openllm
import openai

# Initialize Firestore DB

In [ ]:
# Use the application default credentials.
cred = credentials.ApplicationDefault()

firebase_admin.initialize_app(cred)
db = firestore.client()

# Initialize OpenLLM

In [ ]:
llm = openllm.LLM('NousResearch/Nous-Hermes-llama-2-7b', backend='vllm', torch_dtype='float16', embedded=True)


